<a href="https://colab.research.google.com/github/Mayakshanesht/Autonomous_Driving_Lecture_resources/blob/Perception/02_Define_and_train_the_Discriminator_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [9]:
import tensorflow as tf
import keras
import numpy as np

# Defining the discriminator

In [18]:
def define_discriminator(in_shape= (28, 28, 1)):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters= 64,
                                  input_shape= in_shape,
                                  kernel_size= (3, 3),
                                  strides= (2, 2),
                                  kernel_initializer= 'he_uniform',
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Conv2D(filters= 64,
                                  kernel_size= (3, 3),
                                  strides= (2, 2),
                                  kernel_initializer= 'he_uniform',
                                  padding= 'same'))
    model.add(keras.layers.LeakyReLU())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation= 'sigmoid'))
    
    #opt = keras.optimizers.adam(learning_rate= 0.0002, beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy',
                  optimizer= 'adam',
                  metrics= ['accuracy'])
    
    return model

In [19]:
model = define_discriminator()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3136)             

# Generating the data

In [20]:
def load_data():
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    X_train = np.expand_dims(X_train, axis= -1).astype('float32') / 255.0
    return X_train

In [21]:
def generate_real_example(n_samples):
    dataset = load_data()
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

In [23]:
def generate_fake_examples(n_samples):
    X = np.random.rand(28 * 28 * n_samples)
    X = X.reshape((n_samples, 28, 28, 1))
    y = np.zeros((n_samples, 1))
    return X, y

## Train Synthesizer

In [25]:
def train_dis(model, batch_size= 128, n_iter= 100):
    half_batch = int(batch_size / 2)
    
    for i in range(n_iter):
        X_real, y_real = generate_real_example(half_batch)
        X_fake, y_fake = generate_fake_examples(half_batch)
        
        model.train_on_batch(X_real, y_real)
        model.train_on_batch(X_fake, y_fake)

## Training the model

In [26]:
model = define_discriminator()
train_dis(model= model)

11493376/11490434 [==============================] - 0s 0us/step


# Evaluating the model

In [27]:
X_real, y_real = generate_real_example(64)
X_fake, y_fake = generate_fake_examples(64)

print('Fake data', model.evaluate(X_fake, y_fake, verbose= 0))
print('Real Data', model.evaluate(X_real, y_real, verbose= 0))

Fake data [1.4666251445305534e-05, 1.0]
Real Data [0.00040300076943822205, 1.0]
